In [1]:
!pip install -r ../requirements.txt

  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of tabula-py to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of kubernetes to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple ver

In [1]:
import dotenv
assert dotenv.load_dotenv()

In [2]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
MODEL_ID5 = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
MODEL_ID4 = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"
MODEL_ID6 = "anthropic.claude-3-5-sonnet-20241022-v2:0"

HEADERS = {
    # "anthropic-beta": "max-tokens-3-5-sonnet-2024-07-15",
    "Content-Type": "application/json"
}



# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0, "max_tokens": 81920, 'top_p': 0.9, 'top_k': 100}) # 
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID4, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [3]:
import tools.csv_handling as csv_tools
import tools.database as db_tools
import tools.web_crawl as web_tools
import tools.data_viz as viz_tools
import tools.pdf_handling as pdf_tools
# import tools.stats as stats_tools
import tools.stats_analysis as stats_analysis_tools
# import tools.reasoning as reasoning_tools

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
tools_researcher = [db_tools.query_database, db_tools.get_possible_answers_to_question, db_tools.get_questions_of_given_type]

tools_chart = [viz_tools.custom_plot_from_string_to_s3]

tools_web = [web_tools.get_unesco_data, web_tools.crawl_subpages, web_tools.scrape_text, web_tools.duckduckgo_search] 

tools_file = [csv_tools.process_first_sheet_to_json_from_url, csv_tools.extract_table_from_url_to_string_with_auto_cleanup]

tools_pdf = [pdf_tools.extract_top_paragraphs_from_url]

# tools_stats = [stats_tools.analyze_pirls_data]

tools_stats_analysis = [stats_analysis_tools.calculate_pearson_multiple, stats_analysis_tools.calculate_quantile_regression_multiple]

# tools_reasoning = [reasoning_tools.generate_sub_questions]

tools = tools_researcher + tools_chart + tools_stats_analysis + tools_file + tools_web + tools_pdf # + tools_stats_analysis + tools_reasoning

In [5]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# Step 1: Basic State Definition
class BasicState(TypedDict):
    count: int

# Step 2: More Complex State
class ComplexState(TypedDict):
    count: int
    messages: Annotated[list[HumanMessage | AIMessage], add_messages]

# Step 3: State Modification Functions
def increment_count(state: BasicState) -> BasicState:
    return BasicState(count=state["count"] + 1)

def add_message(state: ComplexState, message: str, is_human: bool = True) -> ComplexState:
    new_message = HumanMessage(content=message) if is_human else AIMessage(content=message)
    return ComplexState(
        count=state["count"],
        messages=state["messages"] + [new_message]
    )

# Step 4: Simple Graph with State
def create_simple_graph():
    workflow = StateGraph(BasicState)
    
    def increment_node(state: BasicState):
        return {"count": state["count"] + 1}
    
    workflow.add_node("increment", increment_node)
    workflow.set_entry_point("increment")
    workflow.add_edge("increment", END)
    
    return workflow.compile()

# Step 5: More Complex Graph with State
def create_complex_graph():
    workflow = StateGraph(ComplexState)
    
    def process_message(state: ComplexState):
        last_message = state["messages"][-1].content if state["messages"] else "No messages yet"
        response = f"Received: {last_message}. Count is now {state['count'] + 1}"
        return {
            "count": state["count"] + 1,
            "messages": state["messages"] + [AIMessage(content=response)]
        }
    
    workflow.add_node("process", process_message)
    workflow.set_entry_point("process")
    workflow.add_edge("process", END)
    
    return workflow.compile()

# Interactive Session
def run_interactive_session():
    print("Welcome to the Interactive LangGraph State Lesson!")
    
    print("\nStep 1: Basic State")
    basic_state = BasicState(count=0)
    print(f"Initial basic state: {basic_state}")
    
    print("\nStep 2: More Complex State")
    complex_state = ComplexState(count=0, messages=[])
    print(f"Initial complex state: {complex_state}")
    
    print("\nStep 3: State Modification")
    modified_basic = increment_count(basic_state)
    print(f"Modified basic state: {modified_basic}")
    
    modified_complex = add_message(complex_state, "Hello, LangGraph!")
    print(f"Modified complex state: {modified_complex}")
    
    print("\nStep 4: Simple Graph with State")
    simple_graph = create_simple_graph()
    result = simple_graph.invoke(BasicState(count=0))
    print(f"Simple graph result: {result}")
    
    print("\nStep 5: Complex Graph with State")
    complex_graph = create_complex_graph()
    initial_state = ComplexState(count=0, messages=[HumanMessage(content="Hello, LangGraph!")])
    result = complex_graph.invoke(initial_state)
    print(f"Complex graph result: {result}")

if __name__ == "__main__":
    run_interactive_session()

Welcome to the Interactive LangGraph State Lesson!

Step 1: Basic State
Initial basic state: {'count': 0}

Step 2: More Complex State
Initial complex state: {'count': 0, 'messages': []}

Step 3: State Modification
Modified basic state: {'count': 1}
Modified complex state: {'count': 0, 'messages': [HumanMessage(content='Hello, LangGraph!')]}

Step 4: Simple Graph with State
Simple graph result: {'count': 1}

Step 5: Complex Graph with State
Complex graph result: {'count': 1, 'messages': [HumanMessage(content='Hello, LangGraph!', id='01105421-cfdf-4ad2-86b5-be3e91532f47'), AIMessage(content='Received: Hello, LangGraph!. Count is now 1', id='eacd608e-b832-4098-99b1-303b1e90bfcd')]}


In [6]:
from typing import TypedDict, Annotated, Optional, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import tool
from bs4 import BeautifulSoup
from sqlalchemy import text
from static.util import ENGINE
import requests
import json

# Update the SearchState to include query_type
class SearchState(TypedDict):
    """
    TypedDict to represent the state used in the search and scraping process.

    Attributes:
        messages (list): A list of messages exchanged between the user and AI.
        query (str): The search query provided by the user.
        results (list): A list containing raw search results.
        relevant_links (List[str]): The top 3 most relevant non-PDF links from the search results.
        query_type (str): Classification of the query as 'out of scope', 'closed question', or 'open question'.
    """
    messages: Annotated[list[HumanMessage | AIMessage], add_messages]
    query: str
    results: list
    relevant_links: List[str]
    query_type: str

def evaluate_query_scope(query: str) -> str:
    """
    Uses an LLM to classify the user's query as either out of scope, a closed question, or an open question.

    Args:
        query (str): The user's search query.

    Returns:
        str: The classification of the query, either "out of scope", "closed question", or "open question".
    """
    prompt = """
    You are a smart assistant that classifies search queries for PIRLS 2021. Given the query below, determine if it is:
    - Out of Scope: The query is unrelated to supported topics or cannot be addressed by this system.
    - Closed Question: The query is likely answerable with a brief, factual response.
    - Open Question: The query requires a detailed response, possibly including opinion or analysis.

    Query: "{query}"

    Respond with one of the following: "out of scope", "closed question", or "open question".
    """

    # Format the prompt with the user query
    formatted_prompt = prompt.format(query=query)

    # Call the LLM to get the classification
    response = llm([HumanMessage(content=formatted_prompt)])
    
    # Process and return the LLM's response
    classification = response.content.strip().lower()  # Normalize the output for consistency
    print("LLM classification:", classification)  # Debugging output for classification result

    # Validate the LLM's response to ensure it matches expected output
    if classification in ["out of scope", "closed question", "open question"]:
        return classification
    else:
        return "unknown"  # Fallback if LLM provides an unexpected response

@tool
def query_database(query: str) -> dict:
    """Query the PIRLS postgres database and return the results as a dictionary.

    Args:
        query (str): The SQL query to execute.

    Returns:
        dict: The results of the query as a dictionary where each entry represents a row with column names as keys.
              Also includes a warning if the query lacks record limiters.
    
    Raises:
        Exception: If the query is invalid or encounters an exception during execution.
    """
    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return {"error": f"Wrong query, encountered exception {e}"}

    # Convert results to a list of dictionaries
    result_dict = [dict(row) for row in res]
    
    # Truncate if the result is too large
    max_results = 100  # Arbitrary cap on rows returned
    if len(result_dict) > max_results:
        result_dict = result_dict[:max_results]
        result_dict.append({"note": "Output truncated due to large result size."})

    return {"query": query, "results": result_dict}
    
# Define the DuckDuckGo search tool
@tool
def duckduckgo_search(query: str) -> list:
    """
    Performs a DuckDuckGo search and retrieves the results.

    Args:
        query (str): The search query for which results are to be fetched.

    Returns:
        list: A list of search results, where each result includes the title, link, and snippet.
              Returns an error message as a string if the search fails.
    """
    try:
        # Replace with actual DuckDuckGo API if available
        # Mocked DuckDuckGo wrapper function call
        wrapper = DuckDuckGoSearchAPIWrapper(region="en-us", time="a", max_results=5)
        search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="text")
        results = search.api_wrapper.results(query, max_results=5, source="text")
        return results
    except Exception as e:
        return str(e)

# Tool for scraping text from web pages
@tool
def scrape_text(url: str, target_elements: list = ['p'], **attributes) -> dict:
    """
    Scrapes text content from specified HTML elements on a webpage.

    Args:
        url (str): The URL of the webpage to scrape.
        target_elements (list): A list of HTML tags to target (default is ['p']).
        **attributes: Additional HTML attributes to filter elements (e.g., class_="highlight").

    Returns:
        dict: A dictionary where each key is an HTML tag (e.g., 'p'), and each value is a list of text content 
              from the specified tag. Returns an empty dictionary if the page could not be scraped.
    """
    result = {}
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        for target_element in target_elements:
            texts = [element.get_text() for element in soup.find_all(target_element, **attributes)]
            result[target_element] = texts[:10]  # Limit to first 10 texts for brevity
    return result

def is_valid_url(url: str) -> bool:
    """
    Checks if a URL is accessible and returns a status of 200 OK.

    Args:
        url (str): The URL to validate.

    Returns:
        bool: True if the URL is accessible and returns a 200 OK status, False otherwise.
    """
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def select_top_non_pdf_links(search_results: list) -> List[str]:
    """
    Selects the top 3 non-PDF links from a list of search results.

    Args:
        search_results (list): A list of search results where each result is a dictionary with a 'link' key.

    Returns:
        List[str]: A list of up to 3 non-PDF URLs from the search results.
    """
    non_pdf_links = [result["link"] for result in search_results if "link" in result and not result["link"].endswith(".pdf")]
    return [link for link in non_pdf_links[:3] if is_valid_url(link)]

def perform_search(state: SearchState) -> SearchState:
    """
    Uses an LLM to transform the search query, execute a database query, optionally execute a web search, 
    and update the state with the results.

    Args:
        state (SearchState): The current state, containing the query and any previous messages.

    Returns:
        SearchState: The updated state with database results, optional web search results, and relevant links.
    """
    original_query = state["query"]

    # Step 1: Check if query is out of scope
    if state.get("query_type") == "out of scope":
        new_message = AIMessage(content="The query is out of scope and cannot be processed.")
        return SearchState(
            messages=state["messages"] + [new_message],
            query=original_query,
            results=[],
            relevant_links=[],
            query_type="out of scope"
        )

    # Step 2: Use LLM to optimize the search query
    query_optimization_prompt = """
    You are an assistant that refines search queries for optimal results. Given the user query below, please transform it 
    to be more effective for a web search. Use concise keywords and rephrase as needed to maximize search relevance.
    You only focus on information related to PIRLS 2021 and UNESCO. Your goal is to provide information as a foundation for decisions by policy-makers.
    Only return the search query in your final output.
    
    Example:
    '''
    impact COVID pandemic students reading scores habits PIRLS 2021
    '''

    Original Query: "{query}"

    Optimized Query:
    """
    formatted_prompt = query_optimization_prompt.format(query=original_query)
    response = llm([HumanMessage(content=formatted_prompt)])
    optimized_query = response.content.strip()  # LLM-optimized query

    print("Original Query:", original_query)
    print("Optimized Query:", optimized_query)  # Debugging output to check query transformation

    # Step 3: Generate and execute the SQL query using the LLM
    sql_generation_prompt = f"""
    You answer all queries with the most relevant data available and an explanation how you found it.
        You know that the database has millions of entries. Always limit your queries to return only the necessary data.
        If data is not provided in the dataset (e.g. trend data), stop the database search.
        Reduce the amount of queries to the dataset as much as possible.
        NEVER return more than 300 rows of data.
        ALWAYS use at least LIMIT 150 and at most LIMIT 300 if grouping data by country.
        NEVER use the ROUND function. Instead use the CAST function for queries.
        ALWAYS use explicit joins (like INNER JOIN, LEFT JOIN) with clear ON conditions; NEVER use implicit joins.
        ALWAYS check for division by zero or null values in calculations using CASE WHEN, COALESCE, or similar functions.
        ALWAYS ensure that the ORDER BY clause uses the correct aggregation function if needed
        NEVER overlook the handling of NULL values in CASE statements, as they can affect calculations.
        ALWAYS verify that data type casting is supported by your database and does not truncate important values.
        NEVER assume JOIN conditions are correct without verifying the relationships between tables.
        ALWAYS consider the performance impact of multiple JOIN operations and MAX functions, and use indexing where appropriate.
        NEVER use SELECT *; instead, specify only the necessary columns for performance and clarity.
        ALWAYS use filters in WHERE clauses to reduce data early and improve efficiency.
        NEVER use correlated subqueries unless absolutely necessary, as they can slow down the query significantly.
        ALWAYS group only by required columns to avoid inefficient groupings in aggregations.
        ALWAYS be transparent, when your queries don't return anything meaningful. Not all data is available in the database.
        ALWAYS write queries that return the required end results with as few steps as possible. 
        ALWAYS when trying to find a mean you return the mean value, not a list of values. 
        ALWAYS focus on the highest level data (e.g. global perspective, if open question, national perspective, if specific country requested).
        NEVER query data by country for general questions (e.g. socioeconomic impact unless requested).
        ALWAYS prioritize for reading score queries to include distribution across benchmarks or quantiles.
        NEVER filter out values for a field in your query.
        ALWAYS consider the diversity of the data (well performing education systems, badly performing education systems)
        ALWAYS ensure that all selected columns not in aggregate functions appear in the GROUP BY clause. Use table aliases to avoid ambiguity. Refer to the schema for correct relationships.
        ALWAYS cast to DECIMAL with specified precision using CAST(value AS DECIMAL(p, s)).
        NEVER allow ambiguity in column references.
        NEVER neglect indexing for frequently joined columns
        NEVER use an INNER JOIN if you need to retain all records from a specific table, even if matching records are missing. ALWAYS use LEFT JOIN when unmatched data should still be included in the results
        NEVER use restrictive joins if all records from the primary dataset are required in the results. ALWAYS choose join types (e.g., LEFT JOIN) that ensure comprehensive results when optional data may be missing.
        NEVER include redundant filters that duplicate existing conditions or selections. ALWAYS simplify queries by removing unnecessary filters to streamline execution.
        NEVER assume all fields contain values. ALWAYS use COALESCE or another method to handle potential NULL values, providing default values where appropriate.
        NEVER use inconsistent or unclear aliasing. ALWAYS maintain clear and consistent alias names across queries for readability and maintenance.
        NEVER assume CORR is supported by all SQL databases. Some SQL environments may not support the CORR function. ALWAYS verify that CORR is available in your database. If not, consider calculating correlation manually if it’s unsupported.
        ALWAYS use the alias defined in the WITH clause consistently throughout the query.
        ALWAYS ensure that the GROUP BY clause includes all non-aggregated columns used in the SELECT statement.
        ALWAYS verify that the data type and precision used in the CAST function are appropriate for the expected range of values.
        ALWAYS include all columns in the ORDER BY clause that you want to sort by in the SELECT statement.
        ALWAYS use the COALESCE function to handle potential NULL values in your calculations.
        ALWAYS ensure that aliases used in Common Table Expressions (CTEs) are properly defined within the CTE.
        ALWAYS ensure that join conditions reference the correct columns and aliases.
        IF querying by country, ALWAYS focus on top 10, bottom 10 and outliers.
        ALWAYS get possible answers to questions in the database first before attempting to query data by answer.
        ALWAYS verify that the code you are using is available in the selected table.
        NEVER use complex CASE statements within aggregate functions for calculating percentages.
        
        ## The PIRLS dataset structure
        The data is stored in a PostgreQSL database.

        # Schema and explanation
        Students
        Student_ID: Int (Primary Key) - uniquely identifies student
        Country_ID: Int (Foreign Key) - uniquely identifies student's country
        School_ID: Int (Foreign Key) - uniquely identifies student's school
        Home_ID: Int (Foreign Key) - uniquely identifies student's home

        StudentQuestionnaireEntries
        Code: String (Primary Key) - possible values: ASBG01, ASBG02A, ASBG02B, ASBG03, ASBG04, ASBG05A, ASBG05B, ASBG05C, ASBG05D, ASBG05E, ASBG05F, ASBG05G, ASBG05H, ASBG05I, ASBG05J, ASBG05K, ASBG06, ASBG07A, ASBG07B, ASBG08A, ASBG08B, ASBG09A, ASBG09B, ASBG09C, ASBG09D, ASBG09E, ASBG09F, ASBG09G, ASBG09H, ASBG10A, ASBG10B, ASBG10C, ASBG10D, ASBG10E, ASBG10F, ASBG11A, ASBG11B, ASBG11C, ASBG11D, ASBG11E, ASBG11F, ASBG11G, ASBG11H, ASBG11I, ASBG11J, ASBR01A, ASBR01B, ASBR01C, ASBR01D, ASBR01E, ASBR01F, ASBR01G, ASBR01H, ASBR01I, ASBR02A, ASBR02B, ASBR02C, ASBR02D, ASBR02E, ASBR03A, ASBR03B, ASBR03C, ASBR04, ASBR05, ASBR06A, ASBR06B, ASBR07A, ASBR07B, ASBR07C, ASBR07D, ASBR07E, ASBR07F, ASBR07G, ASBR07H, ASBR08A, ASBR08B, ASBR08C, ASBR08D, ASBR08E, ASBR08F
        Question: String - the question
        Type: String - describes the type of the question. There are several questions in each type. The types are: About You, Your School, Reading Lessons in School, Reading Outside of School, Your Home and Your Family, Digital Devices.

        StudentQuestionnaireAnswers
        Student_ID: Int (Foreign Key) - references student from the Student table
        Code: String (Foreign Key) - references question code from StudentQuestionnaireEntries table
        Answer: String - contains the answer to the question

        SchoolQuestionnaireEntries
        Code: String (Primary Key) - possible values: ACBG01, ACBG02, ACBG03A, ACBG03B, ACBG04, ACBG05A, ACBG05B, ACBG06A, ACBG06B, ACBG06C, ACBG07A, ACBG07B, ACBG07C, ACBG08, ACBG09, ACBG10AA, ACBG10AB, ACBG10AC, ACBG10AD, ACBG10AE, ACBG10AF, ACBG10AG, ACBG10AH, ACBG10AI, ACBG10AJ, ACBG10BA, ACBG10BB, ACBG10BC, ACBG10BD, ACBG11A, ACBG11B, ACBG11C, ACBG11D, ACBG11E, ACBG11F, ACBG11G, ACBG11H, ACBG11I, ACBG11J, ACBG11K, ACBG11L, ACBG12A, ACBG12B, ACBG12C, ACBG12D, ACBG12E, ACBG12F, ACBG12G, ACBG12H, ACBG12I, ACBG12J, ACBG13, ACBG14A, ACBG14B, ACBG14C, ACBG14D, ACBG14E, ACBG14F, ACBG14G, ACBG14H, ACBG14I, ACBG14J, ACBG14K, ACBG14L, ACBG14M, ACBG14N, ACBG15, ACBG16, ACBG17, ACBG18A, ACBG18B, ACBG18C, ACBG19, ACBG20, ACBG21A, ACBG21B, ACBG21C, ACBG21D, ACBG21E, ACBG21F
        Question: String - contains content of the question
        Type: String - describes a category of a question. There are several questions in each category. The categories are: Instructional Time, Reading in Your School, School Emphasis on Academic Success, School Enrollment and Characteristics, Students’ Literacy Readiness, Principal Experience and Education, COVID-19 Pandemic, Resources and Technology, School Discipline and Safety

        SchoolQuestionnaireAnswers
        School_ID: Int (Composite Key) - references school from Schools table
        Code: String (Composite Key) - references score code from SchoolQuestionnaireEntries table
        Answer: String - answer to the question from the school

        TeacherQuestionnaireEntries
        Code: String (Primary Key) - possible values: ATBG01, ATBG02, ATBG03, ATBG04, ATBG05AA, ATBG05AB, ATBG05AC, ATBG05AD, ATBG05BA, ATBG05BB, ATBG05BC, ATBG05BD, ATBG05BE, ATBG05BF, ATBG05BG, ATBG05BH, ATBG05BI, ATBG05BJ, ATBG05BK, ATBG06, ATBG07AA, ATBG07AB, ATBG07AC, ATBG07AD, ATBG07AE, ATBG07AF, ATBG07AG, ATBG07BA, ATBG07BB, ATBG07BC, ATBG07BD, ATBG07BE, ATBG07BF, ATBG07BG, ATBG08A, ATBG08B, ATBG08C, ATBG08D, ATBG08E, ATBG09A, ATBG09B, ATBG09C, ATBG09D, ATBG10A, ATBG10B, ATBG10C, ATBG10D, ATBG10E, ATBG10F, ATBG10G, ATBG10H, ATBG10I, ATBG10J, ATBG10K, ATBG10L, ATBG11A, ATBG11B, ATBG11C, ATBG11D, ATBG11E, ATBG11F, ATBG11G, ATBG11H, ATBG11I, ATBG12A, ATBG12B, ATBG12C, ATBG12D, ATBG12E, ATBG12F, ATBR01A, ATBR01B, ATBR02A, ATBR02B, ATBR03A, ATBR03B, ATBR03C, ATBR03D, ATBR03E, ATBR03F, ATBR03G, ATBR03H, ATBR04, ATBR05, ATBR06A, ATBR06B, ATBR06C, ATBR06D, ATBR06E, ATBR07AA, ATBR07AB, ATBR07AC, ATBR07AD, ATBR07BA, ATBR07BB, ATBR07BC, ATBR07BD, ATBR08A, ATBR08B, ATBR08C, ATBR08D, ATBR08E, ATBR08F, ATBR08G, ATBR08H, ATBR09A, ATBR09B, ATBR09C, ATBR09D, ATBR09E, ATBR09F, ATBR09G, ATBR09H, ATBR09I, ATBR10A, ATBR10B, ATBR10C, ATBR10D, ATBR10E, ATBR10F, ATBR10G, ATBR10H, ATBR10I, ATBR10J, ATBR10K, ATBR10L, ATBR11A, ATBR11B, ATBR11C, ATBR11D, ATBR11E, ATBR12A, ATBR12BA, ATBR12BB, ATBR12BC, ATBR12BD, ATBR12C, ATBR12DA, ATBR12DB, ATBR12DC, ATBR12EA, ATBR12EB, ATBR12EC, ATBR12ED, ATBR12EE, ATBR13A, ATBR13B, ATBR13C, ATBR13D, ATBR13E, ATBR14, ATBR15, ATBR16, ATBR17, ATBR17A, ATBR17B, ATBR17C, ATBR18A, ATBR18B, ATBR18C, ATBR18D, ATBR18E, ATBR19
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About You, School Emphasis on Academic Success, School Environment, Being a Teacher of the PIRLS Class, Teaching Reading to the PIRLS Class, Teaching the Language of the PIRLS Test, Reading Instruction and Strategies, Teaching Students with Reading Difficulties, Professional Development, Distance Learning During the COVID-19 Pandemic

        TeacherQuestionnaireAnswers
        Teacher_ID: Int (Foreign Key) - references teacher from Teachers table
        Code: String (Foreign Key) - references score code from TeacherQuestionnaireEntries table
        Answer: String - answer to the question from the teacher

        HomeQuestionnaireEntries
        Code: String (Primary Key) - possible values: ASBH01A, ASBH01B, ASBH01C, ASBH01D, ASBH01E, ASBH01F, ASBH01G, ASBH01H, ASBH01I, ASBH01J, ASBH01K, ASBH01L, ASBH01M, ASBH01N, ASBH01O, ASBH01P, ASBH01Q, ASBH01R, ASBH02A, ASBH02B, ASBH03A, ASBH03B, ASBH03C, ASBH03D, ASBH03E, ASBH03F, ASBH04, ASBH05AA, ASBH05AB, ASBH05B, ASBH06, ASBH07A, ASBH07B, ASBH07C, ASBH07D, ASBH07E, ASBH07F, ASBH07G, ASBH08A, ASBH08B, ASBH08C, ASBH08D, ASBH08E, ASBH08F, ASBH09, ASBH10, ASBH11A, ASBH11B, ASBH11C, ASBH11D, ASBH11E, ASBH11F, ASBH11G, ASBH11H, ASBH12, ASBH13, ASBH14A, ASBH14B, ASBH14C, ASBH15A, ASBH15B, ASBH16, ASBH17A, ASBH17B, ASBH18AA, ASBH18AB, ASBH18BA, ASBH18BB, ASBH18CA, ASBH18CB, ASBH18DA, ASBH18DB, ASBH18EA, ASBH18EB, ASBH18FA, ASBH18FB, ASBH18GA, ASBH18GB, ASBH19, ASBH20A, ASBH20B, ASBH20C, ASBH21A, ASBH21B, ASBH21C, ASBH21D, ASBH22
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: Additional Information, Before Your Child Began Primary/Elementary School, Beginning Primary/Elementary School, COVID-19 Pandemic, Literacy in the Home, Your Child's School

        HomeQuestionnaireAnswers
        Home_ID: Int (Foreign Key)
        Code: String (Foreign Key)
        Answer: String

        CurriculumQuestionnaireEntries
        Code: String (Primary Key) - Possible values: COVID01, COVID02A, COVID02B, COVID02C, COVID02T, GEN01, GEN02A, GEN02B, GEN03A, GEN03B, GEN04A, GEN04B, GEN05, GEN06, GEN06T, GEN07A, GEN07B, GEN08AA, GEN08AB, GEN08AC, GEN08AD, GEN08AE, GEN08AF, GEN08B, GEN08C, GEN08T, GEN09AA, GEN09AB, GEN09BAA, GEN09BAB, GEN09BBA, GEN09BBB, GEN09BCA, GEN09BCB, GEN09BDA, GEN09BDB, GEN09BEA, GEN09BEB, GEN09BFA, GEN09BFB, GEN09BGA, GEN09BGB, GEN09BGT, GEN09BT, GEN10AA, GEN10AB, GEN10AC, GEN10AD, GEN10B, GEN10BT, GEN10CA, GEN10CB, GEN10CBT, GEN10CC, GEN10CD, GEN10CDT, GEN10D, GEN10DT, GEN11AA, GEN11AB, GEN11AC, GEN11ACT, GEN11B, GEN11BT, READ01, READ01TA, READ01TB, READ02A, READ02B, READ02C, READ02T, READ03A, READ03B, READ03BT, READ04, READ04TA, READ04TB, READ05A, READ05B, READ05C, READ05D, READ05E, READ05ET, READ05T, READ06A, READ06AT, READ06B, READ06BA, READ06BB, READ06BC, READ06BD, READ06BE, READ06BET, READ06C, READ06CT, READ07AA, READ07AB, READ07BA, READ07BB, READ07BC, READ07CA, READ07CB, READ07CC, READ07DA, READ07DB, READ07T, READ08A, READ08B, READ08C, READ08D, READ08T, READ09A, READ09B, READ09C, READ09T
        Question: String
        Type: String - describes a type of a question. There are several questions in each type. The types are: About the Fourth Grade Language/Reading Curriculum, Areas of Emphasis in the Language/Reading Curriculum, COVID-19 Pandemic, Curriculum Specifications, Early Childhood Education, Grade Structure and Student Flow, Instructional Materials and Use of Digital Devices, Languages of Instruction, Principal Preparation, Teacher Preparation

        Schools
        School_ID: Int (Primary Key) - uniquely identifies a School
        Country_ID: Int (Foreign Key) - uniquely identifies a country

        Teachers
        Teacher_ID: Int (Primary Key) - uniquely identifies a Teacher
        School_ID: Int (Foreign Key) - uniquely identifies a School

        StudentTeachers
        Teacher_ID: Int (Foreign Key)
        Student_ID: Int (Foreign Key)

        Homes
        Home_ID: Int (Primary Key) - uniquely identifies a Home

        Curricula
        Curriculum_ID: Int (Primary Key)
        Country_ID: Int (Foreign Key)

        StudentScoreEntries
        Code: String (Primary Key) - See below for examples of codes
        Name: String
        Type: String

        StudentScoreResults
        Student_ID: Int (Foreign Key) - references student from Students table
        Code: String (Foreign Key) - references score code from StudentScoreEntries table
        Score: Float - the numeric score for a student

        Benchmarks
        Benchmark_ID: Int (Primary Key) - uniquely identifies benchmark
        Score: Int - the lower bound of the benchmark. Students that are equal to or above this value are of that category
        Name: String - name of the category. Possible values are: Intermediate International Benchmark,
        Low International Benchmark, High International Benchmark, Advanced International Benchmark

        Countries
        Country_ID: Int (Primary Key) - uniquely identifies a country
        Name: String - full name of the country
        Code: String - 3 letter code of the country
        Benchmark: Boolean - boolean value saying if the country was a benchmark country. 
        TestType: String - describes the type of test taken in this country. It's either digital or paper.

        # Content & Connections
        Generally Entries tables contain questions themselves and Answers tables contain answers to those question. 
        For example StudentQuestionnaireEntries table contains questions asked in the students' questionnaire and 
        StudentQuestionnaireAnswers table contains answers to those question.

        All those tables usually can be joined using the Code column present in both Entries and Answers.

        Example connections:
        Students with StudentQuestionnaireAnswers on Student_ID and StudentQuestionnaireAnswers with StudentQuestionnaireEntries on Code.
        Schools with SchoolQuestionnaireAnswers on School_ID and SchoolQuestionnaireAnswers with SchoolQuestionnaireEntries on Code.
        Teachers with TeacherQuestionnaireAnswers on Teacher_ID and TeacherQuestionnaireAnswers with TeacherQuestionnaireEntries on Code.
        Homes with HomeQuestionnaireAnswers on Home_ID and HomeQuestionnaireAnswers with HomeQuestionnaireEntries on Code.
        Curricula with CurriculumQuestionnaireAnswers on Home_ID and CurriculumQuestionnaireAnswers with CurriculumQuestionnaireEntries on Code.

        In the student evaluation process 5 distinct scores were measured. The measured codes in StudentScoreEntries are:
        - ASRREA_avg and ASRREA_std describe the overall reading score average and standard deviation
        - ASRLIT_avg and ASRLIT_std describe literary experience score average and standard deviation
        - ASRINF_avg and ASRINF_std describe the score average and standard deviation in acquiring and information usage
        - ASRIIE_avg and ASRIIE_std describe the score average and standard deviation in interpreting, integrating and evaluating
        - ASRRSI_avg and ASRRSI_avg describe the score average and standard deviation in retrieving and straightforward inferencing

        Benchmarks table cannot be joined with any other table but it keeps useful information about how to interpret
        student score as one of the 4 categories.   

        # Examples

        1) A simple query that answers the question 'What percentage of students in Egypt reached the Low International Benchmark?' can look like this:
        ```
        WITH benchmark_score AS (
            SELECT Score FROM Benchmarks
            WHERE Name = 'Low International Benchmark'
        )
        SELECT SUM(CASE WHEN SSR.score >= bs.Score THEN 1 ELSE 0 END) / COUNT(*)::float as percentage
        FROM Students AS S
        JOIN Countries AS C ON C.Country_ID = S.Country_ID
        JOIN StudentScoreResults AS SSR ON SSR.Student_ID = S.Student_ID
        CROSS JOIN benchmark_score AS bs
        WHERE C.Name = 'Egypt' AND SSR.Code = 'ASRREA_avg'
        ```

        2) A simple query that answers the question 'Which country had an average reading score between 549 and 550 for its students?' can look like this:
        '''
        SELECT C.Name AS Country
        FROM Students as S
        JOIN Countries as C ON S.Country_ID = C.Country_ID
        JOIN StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID
        WHERE SSR.Code = 'ASRREA_avg'
        GROUP BY C.Name
        HAVING AVG(ssr.Score) BETWEEN 549 AND 550;
        '''

    User Intent: "{optimized_query}"

    SQL Query:
    """
    sql_response = llm([HumanMessage(content=sql_generation_prompt)])
    generated_sql_query = sql_response.content.strip()  # LLM-generated SQL query

    print("Generated SQL Query:", generated_sql_query)  # Debugging output for SQL query

    # Execute SQL query and store the result as a dictionary
    sql_result = query_database(generated_sql_query)
    
    # Create a message for the SQL result
    sql_message = AIMessage(content=f"Executed SQL query for PIRLS database: {generated_sql_query}\nResult:\n{sql_result}")

    # Step 4: Optionally perform a web search with optimized query if additional context is required
    search_results, relevant_links = [], []
    if state.get("query_type") == "open question":  # Perform search only for open questions
        search_results = duckduckgo_search(optimized_query)
        
        # Debugging output to check search results
        print("Full Search Results:", search_results)

        if not isinstance(search_results, str):
            # Select top 3 non-PDF links
            relevant_links = select_top_non_pdf_links(search_results)

            # Debugging output to check selected non-PDF links
            print("Top Non-PDF Links:", relevant_links)

    # Return the updated state with SQL and optional web search results
    return SearchState(
        messages=state["messages"] + [sql_message],
        query=optimized_query,
        results=search_results,
        relevant_links=relevant_links,
        query_type=state.get("query_type", "unknown"),
        sql_results=sql_result  # Store SQL result as a dict
    )


def scrape_top_relevant_content(state: SearchState) -> List[str]:
    """
    Scrapes content from each of the top 3 relevant links and uses an LLM to select key quotes.

    Args:
        state (SearchState): The current state containing the top 3 non-PDF relevant links.

    Returns:
        List[str]: A list of up to 3 key quotes, each formatted with a "Read more" link back to its source.
    """
    # Aggregate scraped content from each link
    all_content = []
    for link in state["relevant_links"]:
        scraped_content = scrape_text(link)
        
        # Collect content for LLM processing
        for tag, texts in scraped_content.items():
            for text in texts:
                all_content.append(f"{text.strip()} [Source]({link})")

    # Prepare prompt for LLM to select key quotes
    prompt = """
    You are an assistant analyzing content from multiple web pages related to the query "{query}".
    Please review the provided content and select the top 2-3 most relevant quotes.

    Content:
    {content}

    Provide each quote as a separate bullet point, and include the "Read more" link for context.
    """

    # Format content and prompt
    formatted_content = "\n".join(f"- {entry}" for entry in all_content)
    formatted_prompt = prompt.format(query=state["query"], content=formatted_content)

    # Get the selected quotes from the LLM
    response = llm([HumanMessage(content=formatted_prompt)])
    key_quotes = response.content.splitlines()  # Assuming LLM returns each quote as a separate line

    # Debug output to verify LLM response
    print("LLM-selected key quotes:", key_quotes)

    return key_quotes[:3]  # Limit to the top 3 quotes if LLM returns more

def generate_markdown_with_llm(state: SearchState) -> str:
    """
    Generates a markdown summary using an LLM, based on search query, top links, key quotes, and messages.

    Args:
        state (SearchState): The current state containing the query, relevant links, key quotes, and messages.

    Returns:
        str: A markdown-formatted summary, generated by the LLM.
    """
    # Check if relevant links are available
    if not state.get("relevant_links"):
        return "No relevant links were found for the query."

    # Scrape key quotes from the top 3 relevant links
    key_quotes = scrape_top_relevant_content(state)
    
    # Format messages for the prompt
    messages_str = "\n".join(
        f"{'User' if isinstance(msg, HumanMessage) else 'AI'}: {msg.content}"
        for msg in state["messages"]
    )

    # Construct the prompt for LLM
    prompt = """
    Create a markdown summary for the search query "{query}". 
    ALWAYS write your final output in the style of a data loving and nerdy UNESCO data and statistics team that LOVES minimalist answers that focus on numbers, percentages, plots, correlations and distributions from a global perspective.
    ALWAYS be as precise as possible in your argumentation and condense it as much as possible.
    ALWAYS start the output with a summary in the style of brutal simplicity.
    ALWAYS use unordered lists. NEVER use ordered lists.
    
    Include:
    - The most relevant link with a short description.
    - Key quotes (2-3) with direct links to the source should be interwoven into key findings.
    - A citation section with the link.
    
    Query: {query}
    Relevant Links: {links}
    Key Quotes:
    {quotes}
    """

    formatted_prompt = prompt.format(
        query=state["query"],
        links="\n".join(f"- {link}" for link in state["relevant_links"]),
        quotes="\n".join(f"- \"{quote}\"" for quote in key_quotes),
        messages=messages_str
    )

    # Get the markdown summary from the LLM
    response = llm([HumanMessage(content=formatted_prompt)])
    return response.content

# Initialize the workflow with DuckDuckGo search
def create_search_graph():
    workflow = StateGraph(SearchState)

    # Node for adding an initial user message to request a search
    workflow.add_node("add_user_message", lambda state: SearchState(
        messages=state["messages"] + [HumanMessage(content=f"Please search for: {state['query']}")],
        query=state["query"],
        results=[]
    ))

    # Node for performing the search
    workflow.add_node("search", perform_search)

    # Set the entry point and define the workflow path
    workflow.set_entry_point("add_user_message")
    workflow.add_edge("add_user_message", "search")
    workflow.add_edge("search", END)

    return workflow.compile()

def run_search_session():
    """
    Orchestrates the search session workflow, with an initial LLM-based evaluation of the query type.
    """
    # Initial user input
    query = "Are boys or girls most lagging behind in reading abilities?"

    # Evaluate the query scope using LLM
    query_type = evaluate_query_scope(query)
    print("Query Type:", query_type)  # Output the evaluation result

    # Initialize the state with the query type included
    initial_state = SearchState(messages=[], query=query, results=[], relevant_links=[], query_type=query_type)

    # Check if the query is out of scope
    if query_type == "out of scope":
        print("The query is out of scope and cannot be processed.")
        return

    # Run the search workflow
    search_graph = create_search_graph()
    final_state = search_graph.invoke(initial_state)
    
    # Generate markdown output with the LLM and print
    markdown_output = generate_markdown_with_llm(final_state)
    print(markdown_output)

if __name__ == "__main__":
    run_search_session()

/tmp/ipykernel_19963/2361500785.py:57: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm([HumanMessage(content=formatted_prompt)])


LLM classification: open question
Query Type: open question
Original Query: Are boys or girls most lagging behind in reading abilities?
Optimized Query: PIRLS 2021 gender gap reading performance UNESCO analysis
Generated SQL Query: To address the user's intent regarding the PIRLS 2021 gender gap in reading performance, I'll need to query the database for reading scores by gender across countries. Here's a query that will provide this information:

```sql
WITH gender_scores AS (
    SELECT 
        C.Name AS Country,
        SQA.Answer AS Gender,
        AVG(SSR.Score) AS Avg_Reading_Score
    FROM Students S
    INNER JOIN Countries C ON S.Country_ID = C.Country_ID
    INNER JOIN StudentQuestionnaireAnswers SQA ON S.Student_ID = SQA.Student_ID
    INNER JOIN StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID
    INNER JOIN StudentQuestionnaireEntries SQE ON SQA.Code = SQE.Code
    WHERE SQE.Code = 'ASBG01' -- This is the code for the gender question
    AND SSR.Code = 'ASRREA_avg

/tmp/ipykernel_19963/2361500785.py:431: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  sql_result = query_database(generated_sql_query)


Full Search Results: [{'snippet': 'Trends in Average Achievement by Gender. Exhibit 2.3 contains the trend results by gender for the 43 countries that assessed fourth grade students at the same time of year as in previous assessments. Although 21 countries had lower average achievement in 2021 than in 2016, for the most part the decreases in achievement were similar for girls ...', 'title': 'Results by Gender - Trends in Reading Acheivement - PIRLS 2021', 'link': 'https://pirls2021.org/results/trends/by-gender/'}, {'snippet': "Conducted every five years since 2001, PIRLS is recognized as the global standard for assessing trends in reading achievement at the fourth grade. PIRLS 2021 was the fifth assessment cycle, providing 20 years of trend results. PIRLS and TIMSS are directed by IEA's TIMSS & PIRLS International Study Center at Boston College in close cooperation ...", 'title': 'PIRLS 2021 International Results in Reading', 'link': 'https://pirls2021.org/results'}, {'snippet': 'Group

In [16]:
def query_database(query: str) -> dict:
    """Query the PIRLS postgres database and return the results as a dictionary.

    Args:
        query (str): The SQL query to execute.

    Returns:
        dict: The results of the query as a dictionary where each entry represents a row with column names as keys.
              Also includes a warning if the query lacks record limiters.
    
    Raises:
        Exception: If the query is invalid or encounters an exception during execution.
    """
    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return {"error": f"Wrong query, encountered exception {e}"}

    # Convert results to a list of dictionaries
    result_dict = [dict(row) for row in res]
    
    # Truncate if the result is too large
    max_results = 100  # Arbitrary cap on rows returned
    if len(result_dict) > max_results:
        result_dict = result_dict[:max_results]
        result_dict.append({"note": "Output truncated due to large result size."})

    return {"query": query, "results": result_dict}

In [21]:
query = """
WITH gender_scores AS (
    SELECT 
        C.Name AS Country,
        SQA.Answer AS Gender,
        AVG(SSR.Score) AS Avg_Reading_Score
    FROM Students S
    INNER JOIN Countries C ON S.Country_ID = C.Country_ID
    INNER JOIN StudentQuestionnaireAnswers SQA ON S.Student_ID = SQA.Student_ID
    INNER JOIN StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID
    INNER JOIN StudentQuestionnaireEntries SQE ON SQA.Code = SQE.Code
    WHERE SQE.Code = 'ASBG01' -- Assuming this is the code for gender
    AND SSR.Code = 'ASRREA_avg' -- Overall reading score
    GROUP BY C.Name, SQA.Answer
)
SELECT 
    Country,
    MAX(CASE WHEN Gender = 'Girl' THEN Avg_Reading_Score END) AS Girls_Score,
    MAX(CASE WHEN Gender = 'Boy' THEN Avg_Reading_Score END) AS Boys_Score,
    MAX(CASE WHEN Gender = 'Girl' THEN Avg_Reading_Score END) - 
    MAX(CASE WHEN Gender = 'Boy' THEN Avg_Reading_Score END) AS Gender_Gap
FROM gender_scores
GROUP BY Country
ORDER BY Gender_Gap DESC
LIMIT 150;
"""

In [22]:
query_database(query)

ValueError: dictionary update sequence element #0 has length 12; 2 is required

In [111]:
# Initialize state with a query and empty messages/results
initial_state = SearchState(messages=[], query="What is the global trend in fourth grade reading achievement as revealed by the PIRLS 2021 results?", results=[])

# Run the workflow
search_graph = create_search_graph()
final_state = search_graph.invoke(initial_state)

# Format the results as markdown using LLM and print
markdown_output = format_results_as_markdown_with_llm(final_state)
print(markdown_output)

Here's a markdown summary based on the search results:

### [PDF PIRLS 2021 International Results in Reading](https://pirls2021.org/wp-content/uploads/2022/files/PIRLS-2021-International-Results-in-Reading.pdf)

PIRLS 2021 was the fifth assessment cycle, providing 20 years of trend results in reading achievement at the fourth grade level. It serves as a global standard for assessing these trends.

### [Results - Countries' Reading Achievement - PIRLS 2021](https://pirls2021.org/results/achievement/overall/)

The PIRLS 2021 study included 57 participating countries and 8 benchmarking participants. Results for 43 countries and 5 benchmarking participants that collected data at the end of fourth grade are presented, showing average reading achievement and scale score distributions.

### [PIRLS 2021 | IEA.nl](https://www.iea.nl/studies/iea/pirls/2021)

PIRLS 2021 is the fifth cycle in the PIRLS assessment, providing internationally comparative data on fourth-grade students' reading achieve

In [112]:
final_state["link_list"]

['https://pirls2021.org/wp-content/uploads/2022/files/PIRLS-2021-International-Results-in-Reading.pdf',
 'https://pirls2021.org/results/achievement/overall/',
 'https://www.iea.nl/studies/iea/pirls/2021']

In [43]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Annotated
from operator import add
from sqlalchemy import text
from static.util import ENGINE
from typing import Literal
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage


# Define the state structure
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add]  # Accumulate messages using operator.add
    query_results: dict  # Store the structured query results

class SQLAgent:
    def __init__(self, model, tools, system_prompt=""):
        self.system_prompt = system_prompt

        # Initializing the graph with AgentState
        graph = StateGraph(AgentState)

        # Adding nodes
        graph.add_node("llm", self.call_llm)
        graph.add_node("function", self.execute_function)
        graph.add_conditional_edges(
            "llm",
            self.exists_function_calling,
            {True: "function", False: END}
        )
        graph.add_edge("function", "llm")

        # Setting the entry point
        graph.set_entry_point("llm")

        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_function_calling(self, state: AgentState):
        result = state['messages'][-1]
        return hasattr(result, 'tool_calls') and len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system_prompt:
            messages = [SystemMessage(content=self.system_prompt)] + messages
        message = self.model.invoke(messages)
        print(f"LLM Response: {message.content}")
        return {'messages': state['messages'] + [message]}  # Append LLM response to messages

    def execute_function(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            if t['name'] not in self.tools:
                result = "Error: Invalid tool name, retrying..."
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print(f"Tool Results: {results}")
        return {'messages': state['messages'] + results}  # Append tool execution results

    def run(self, initial_messages):
        try:
            # Execute the graph and retrieve the final state
            final_state = self.graph.invoke({"messages": [HumanMessage(content=initial_messages)], "query_results": {}}, {"recursion_limit": 50})

            # Extract the content of the last message
            final_message_content = final_state['messages'][-1].content

            # Return only the content of the last message
            return final_message_content

        except Exception as e:
            return (
                "**I'm sorry, but I can't process your request regarding PIRLS 2021 data right now because the server is currently unreachable. "
                "Please try again later.**\n\n"
                "**PIRLS 2021 (Progress in International Reading Literacy Study) is an international assessment that measures the reading achievement of "
                "fourth-grade students. Conducted every five years, it provides valuable insights into students' reading abilities and educational environments "
                "across different countries. For more information, you can visit the PIRLS 2021 website.**\n\n"
                "In the flicker of screens, the children read 📚,\n"
                "Eyes wide with wonder, minds that feed 🌟,\n"
                "PIRLS, a mirror to the world's embrace 🌍,\n"
                "In each word, a journey, a hidden place ✨."
            )

In [ ]:
model = llm
doc_agent = SQLAgent(model, tools, system_prompt=prompt)

In [41]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


# Define the schema for the input
class InputState(TypedDict):
    question: str


# Define the schema for the output
class OutputState(TypedDict):
    answer: str


# Define the overall schema, combining both input and output
class OverallState(InputState, OutputState):
    pass


# Define the node that processes the input and generates an answer
def answer_node(state: InputState):
    # Example answer and an extra key
    return {"answer": "bye", "question": state["question"]}


# Build the graph with input and output schemas specified
builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(answer_node)  # Add the answer node
builder.add_edge(START, "answer_node")  # Define the starting edge
builder.add_edge("answer_node", END)  # Define the ending edge
graph = builder.compile()  # Compile the graph

# Invoke the graph with an input and print the result
print(graph.invoke({"question": "hi"}))

{'answer': 'bye'}
